In [2]:
import networkx as nx
import random
from collections import deque
from random import randrange
import cnfgen
from pysat.solvers import Solver
import numpy as np


import glob
import numpy as np
from scipy.sparse import load_npz
class GraphDataset(object):
    def __init__(self,folder_path,ordered=False):
        super().__init__()

        self.file_paths=glob.glob(f'{folder_path}/*.npz')
        self.file_paths.sort()
        self.ordered=ordered

        if self.ordered:
            self.i = 0

    def __len__(self):
        return len(self.file_paths)
    
    def get(self):
        if self.ordered:
            file_path = self.file_paths[self.i]
            self.i = (self.i + 1)%len(self.file_paths)
        else:
            file_path = random.sample(self.file_paths, k=1)[0]
        return load_npz(file_path).toarray()

In [17]:
# import networkx as nx
# import random
# from collections import deque


# dataset=GraphDataset('../data_color/validation/3col_50vertices')
# number_of_colors = 3

# for _ in range(len(dataset)):
# # graph = nx.erdos_renyi_graph(n=100, p=0.1)


# # n=50
# # G=nx.Graph()
# satisfiable=True
# while satisfiable:
#     u,v = np.random.randint(0, n, 2)
#     if u==v:
#         continue
#     else:
#         G.add_edge(u, v)
#         formula=cnfgen.GraphColoringFormula(G,colors=3)
#         s = Solver(bootstrap_with=formula.clauses())
#         satisfiable=s.solve()
#         if not satisfiable:
#             # break
#             G.remove_edge(u,v)
# graph=G
# solution = {}
# # for i in range(graph.number_of_nodes()):
# for i in graph.nodes():
#     solution[i] = random.randint(1, number_of_colors)

# initial_conflict_count = sum(solution[u] == solution[v] for u, v in graph.edges())
# print("Initial conflict count:", initial_conflict_count)

# tabu_size=5
# tabu=deque()

# max_iterations = 1000
# curr_conflict_count=initial_conflict_count
# best_conflict_count=initial_conflict_count
# for _ in range(max_iterations):
#     if curr_conflict_count==0:
#         print('Found a valid coloring')
#         break
#     # best_delta = float('-inf')
#     # best_color = None
#     # best_node = None

#     move_candidates=[]

#     for node in graph.nodes():
#         # delta_1 = sum(solution[neighbor] == solution[node] for neighbor in graph.neighbors(node))

#         for color in range(1, number_of_colors + 1):
            
#             if color != solution[node]:
#                 new_solution=solution.copy()
#                 new_solution[node]=color
#                 new_conflict_count=sum(new_solution[u] == new_solution[v] for u, v in graph.edges())
#                 # delta_2 = sum(color == solution[neighbor] for neighbor in graph.neighbors(node))

#                 # total_delta = delta_2 - delta_1
#                 # move_candidates.append((total_delta,node,color))
#                 move_candidates.append((new_conflict_count,node,color))
#     move_candidates.sort()
#     # print(move_candidates)

#     # for delta,node,color in move_candidates:
#     for new_conflict,node,color in move_candidates:
#         # if curr_conflict_count+delta<best_conflict_count or (node,color) not in tabu:
#         if new_conflict<best_conflict_count or (node,color) not in tabu:
#             solution[node]=color
#             curr_conflict_count= new_conflict
#             tabu.append((node,color))
#             break
#         else:
#             pass
#             # print('Tabu move')

#     best_conflict_count=min(best_conflict_count,curr_conflict_count)

#     if len(tabu) > tabu_size:  # queue full
#         tabu.popleft() 
    
# print('curr_conflict_count:',curr_conflict_count)
# conflict_count = sum(solution[u] == solution[v] for u, v in graph.edges())
# print("Final conflict count:", conflict_count)


Initial conflict count: 31
curr_conflict_count: 7
Final conflict count: 7


# Greedy

In [19]:
import networkx as nx
import random

dataset=GraphDataset('../data_color/validation/3col_300vertices')
number_of_colors = 3

mean=0

solved=0



for _ in range(len(dataset)):
    graph=nx.from_numpy_array(dataset.get()) 

    solution = {}
    for i in range(graph.number_of_nodes()):
        # solution[i] = random.randint(1, number_of_colors)
        solution[i] = 1

    

    conflict_count = sum(solution[u] == solution[v] for u, v in graph.edges())
    print("Initial conflict count:", conflict_count)
    print("Number of edges in the graph",graph.number_of_edges())

    max_iterations = 1000
    actions_taken=set([])
    for _ in range(max_iterations):
        best_delta = float('-inf')
        best_color = None
        best_node = None

        for node in range(graph.number_of_nodes()):
            # if node not in actions_taken:
            if True:
                delta_1 = sum(solution[neighbor] == solution[node] for neighbor in graph.neighbors(node))

                for color in range(1, number_of_colors + 1):
                    if color != solution[node]:
                        delta_2 = sum(color == solution[neighbor] for neighbor in graph.neighbors(node))

                        total_delta = delta_1 - delta_2
                        if total_delta > best_delta:
                            best_delta = total_delta
                            best_color = color
                            best_node = node

        if best_delta > 0:
            solution[best_node] = best_color
            actions_taken.add(best_node)
        else:
            break

    conflict_count = sum(solution[u] == solution[v] for u, v in graph.edges())
    mean-=conflict_count
    print(solution)
    print("Final conflict count:", conflict_count)
    if conflict_count==0:
        solved+=1
    

print('Test score:',mean/len(dataset))
print('Solved',solved)

Initial conflict count: 673
Number of edges in the graph 673
{0: 3, 1: 2, 2: 3, 3: 2, 4: 3, 5: 1, 6: 3, 7: 3, 8: 2, 9: 1, 10: 3, 11: 2, 12: 3, 13: 2, 14: 3, 15: 2, 16: 1, 17: 1, 18: 1, 19: 2, 20: 2, 21: 1, 22: 1, 23: 3, 24: 1, 25: 1, 26: 1, 27: 3, 28: 2, 29: 3, 30: 2, 31: 1, 32: 2, 33: 1, 34: 3, 35: 1, 36: 2, 37: 3, 38: 1, 39: 3, 40: 2, 41: 1, 42: 3, 43: 2, 44: 1, 45: 1, 46: 2, 47: 2, 48: 1, 49: 2, 50: 1, 51: 1, 52: 1, 53: 2, 54: 1, 55: 2, 56: 2, 57: 1, 58: 3, 59: 1, 60: 1, 61: 3, 62: 1, 63: 3, 64: 2, 65: 1, 66: 2, 67: 3, 68: 3, 69: 1, 70: 3, 71: 3, 72: 3, 73: 3, 74: 2, 75: 1, 76: 3, 77: 2, 78: 3, 79: 1, 80: 3, 81: 2, 82: 1, 83: 3, 84: 1, 85: 3, 86: 1, 87: 1, 88: 2, 89: 1, 90: 2, 91: 2, 92: 1, 93: 3, 94: 2, 95: 3, 96: 3, 97: 2, 98: 3, 99: 1, 100: 2, 101: 3, 102: 1, 103: 1, 104: 1, 105: 3, 106: 1, 107: 1, 108: 1, 109: 2, 110: 1, 111: 2, 112: 1, 113: 1, 114: 3, 115: 1, 116: 3, 117: 2, 118: 2, 119: 2, 120: 3, 121: 2, 122: 1, 123: 2, 124: 2, 125: 3, 126: 3, 127: 2, 128: 2, 129: 1, 130: 1, 